In [63]:
final_merged_df_cleaned.describe()

,isAdult,runtimeMinutes,seasonNumber,episodeNumber,averageRating,numVotes
count,411864.000000,411864.000000,411864.000000,411864.000000,411864.000000,411864.000000
mean,0.000415,38.513621,4.249629,34.267732,7.400160,304.485464
std,0.020372,22.167202,18.375812,361.300968,0.996325,1783.460731
min,0.000000,1.000000,1.000000,0.000000,1.000000,5.000000
25%,0.000000,23.000000,1.000000,4.000000,6.900000,14.000000
50%,0.000000,37.000000,2.000000,8.000000,7.500000,35.000000
75%,0.000000,47.000000,5.000000,15.000000,8.000000,139.000000
max,1.000000,1845.000000,2017.000000,15191.000000,10.000000,258144.000000


In [ ]:
def read_imdb_data(path):
    df = pd.read_csv(path)
    df['averageRating'] = round(df['averageRating'], 1)

    featlst = list(df.columns)

    for feat in featlst:
        l = df[feat]  

        if feat == 'isAdult':
            processed = [1 if v == 1 else 0 for v in l]
            df[feat] = processed

        if feat == 'runtimeMinutes':
            processed = [0 if v <= 23 else 1 if v <= 37 else 2 if v <= 47 else 3 for v in l]
            df[feat] = processed

        if feat == 'seasonNumber':
            processed = [0 if v <= 1 else 1 if v <= 2 else 2 if v <= 5 else 3 for v in l]
            df[feat] = processed

        if feat == 'episodeNumber':
            processed = [0 if v <= 4 else 1 if v <= 8 else 2 if v <= 15 else 3 for v in l]
            df[feat] = processed

        if feat == 'numVotes':
            processed = [0 if v <= 14 else 1 if v <= 35 else 2 if v <= 139 else 3 for v in l]
            df[feat] = processed

    return df



In [ ]:
def read_data(path):
    
    df=pd.read_csv(path)
    l=[]
    for v in df['credit'].values:
        if v==2.0:
            l.append(0)
        else:
            l.append(1)
    df['credit']=l

    orig_df=df


    featlst=list(df.columns)
    featlst.remove('credit')


    for feat in featlst:
        highest=max(df[feat])
        lowest=min(df[feat])
        #if len(list(set(df[feat])))>4:
        #    print (feat,len(list(set(df[feat]))))
        l=df[feat]
        if feat=='month':
            processed=[]
            for v in l:
                if v<=10:
                    processed.append(0)
                elif v<15:
                    processed.append(1)
                elif v<25:
                    processed.append(2)
                else:
                    processed.append(3)
            df[feat]=processed
        if feat=='housing':
            print (feat)
            processed=[]
            for v in l:
                if v==1:
                    processed.append(1)
                else:
                    processed.append(0)
            df[feat]=processed
        
        if feat=='installment_plans' or feat=='number_of_credits':
            print (feat)
            processed=[]
            for v in l:
                if v<=1:
                    processed.append(0)
                else:
                    processed.append(1)
            df[feat]=processed
        if feat=='employment' or feat=='credit_history' or feat=='skill_level':
            print (feat)
            processed=[]
            for v in l:
                if v<=1:
                    processed.append(0)
                else:
                    processed.append(v)
            df[feat]=processed
            
        if feat=='credit_amount':
            processed=[]
            for v in l:
                if v<=1500:
                    processed.append(0)
                elif v<3000:
                    processed.append(1)
                elif v<5500:
                    processed.append(2)
                else:
                    processed.append(3)
            df[feat]=processed
        if feat=='purpose':
            l=[]
            for v in df['purpose']:
                if v>4:
                    l.append(5)
                elif v<=2:
                    l.append(2)
                else:
                    l.append(v)
            df[feat]=l
        if feat=='other_debtors':
            processed=[]
            for v in l:
                if v<=0:
                    processed.append(0)
                else:
                    processed.append(1)
            df[feat]=processed
        if feat=='savings':
            processed=[]
            for v in l:
                if v<=0:
                    processed.append(0)
                elif v<=3:
                    processed.append(1)
                else:
                    processed.append(2)
            df[feat]=processed